In [ ]:
import datetime as dt
from datetime import timedelta
from airflow import DAG
from airflow.operators.bash import BashOperator
from airflow.operators.python import PythonOperator
import pandas as pd

In [ ]:
def CSVToJson(file_path, col):
    df=pd.read_csv(file_path)
    for i, r in df.iterrows():
        print(r[col])
    
    df.to_json("fromAirFlow.json", orient="records")
CSVToJson("mycsv.csv", "name")

In [ ]:
default_args={
    "owner": "dina",
    "start_date": dt.datetime(2023,7,27),
    "retries": 1,
    "retry_delay": dt.timedelta(minutes=5)
}

with DAG('MyCSVDAG',
          default_args=default_args,
          schedule= timedelta(minutes=5),
          # " 0 * * * * ", 
        ) as dag:
    print_starting = BashOperator(task_id="starting",
                                  bash_command='echo "I am Reading the CSV now ..."'
                                  )
    CSVJson= PythonOperator(task_id="Converting_CSV_to_Json",
                            python_callable=CSVToJson
                            )
    print_starting.set_downstream(CSVJson)